In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.opindia.com/latest-news/"
data = requests.get(url)

In [3]:
data

<Response [200]>

In [5]:
#data.content

In [6]:
soup = BeautifulSoup(data.content,"html.parser")

In [7]:
def fetch_article(url):
    data = requests.get(url)
    soup = BeautifulSoup(data.content,"html.parser")
    articles = []
    
    for i in soup.find_all("h3",class_ = ["entry-title td-module-title"]):
        articles.append(i.find('a')['title'])
    return articles

In [8]:
urllist = []
for i in range(2,21,1):
    url = "https://www.opindia.com/latest-news/page/" + str(i) + "/"
    urllist.append(url)

In [10]:
#urllist

In [11]:
all_articles = []
for i in urllist:
    all_articles.extend(fetch_article(i))

In [12]:
p_art = []
for i in all_articles:
    q = i.upper()
    
    import re
    q = re.sub("[^A-Z0-9]","",q)
    
    from nltk.stem import PorterStemmer
    tk_q = q.split(" ")
    sent = ""
    
    for j in tk_q:
        ps = PorterStemmer()
        sent = sent + " " + ps.stem(j).upper()
    p_art.append(sent)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
A = tf.fit_transform(p_art).toarray()

In [14]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=5)
cl_res = km.fit(A)

In [15]:
cl_res.labels_

array([3, 1, 1, ..., 1, 1, 1])

In [16]:
import pandas as pd

In [17]:
Q = pd.DataFrame(p_art,columns=["Article"])
Q['Cluster']=cl_res.labels_

In [18]:
Q.head()

,Article,Cluster
0,THEATLANTICFEARMONGERSABOUTHINDUISATIONOFINDI...,3
1,MANEKAGANDHISUPPORTSIASCOUPLEACCUSEDOFMISUSIN...,1
2,KARNATAKAHIJABROWRESURFACESMANGALOREUNIVERSIT...,1
3,JAMIATULAMAIHINDRAISESGYANVAPIISSUEATDEOBANDE...,1
4,CANTDOANYTHINGMONKEYSAREHANUMANINCARNATIONAND...,1


In [19]:
Q

,Article,Cluster
0,THEATLANTICFEARMONGERSABOUTHINDUISATIONOFINDI...,3
1,MANEKAGANDHISUPPORTSIASCOUPLEACCUSEDOFMISUSIN...,1
2,KARNATAKAHIJABROWRESURFACESMANGALOREUNIVERSIT...,1
3,JAMIATULAMAIHINDRAISESGYANVAPIISSUEATDEOBANDE...,1
4,CANTDOANYTHINGMONKEYSAREHANUMANINCARNATIONAND...,1
...,...,...
1971,ELONMUSKMOCKSWOKESAGAINAFTERTWITTERTAKEOVERCA...,1
1972,MUSLIMPERSONALLAWBOARDSAYSUNIFORMCIVILCODEISU...,1
1973,WOULDHAVEATTACKEDPMMODITOOIFHEHADCHALLENGEDUS...,1
1974,CONGRESSPANELRECOMMENDSSUSPENSIONOFEXPPCCCHIE...,1


In [20]:
E = {1 : "politics",
2 : "religion",
3 : "sports",
4 : "entertainment",
0 : "geopolitics"}

In [21]:
R = []
for i in Q.Cluster:
    R.append(E[i])
Q['category'] = R

In [22]:
Q.head()

,Article,Cluster,category
0,THEATLANTICFEARMONGERSABOUTHINDUISATIONOFINDI...,3,sports
1,MANEKAGANDHISUPPORTSIASCOUPLEACCUSEDOFMISUSIN...,1,politics
2,KARNATAKAHIJABROWRESURFACESMANGALOREUNIVERSIT...,1,politics
3,JAMIATULAMAIHINDRAISESGYANVAPIISSUEATDEOBANDE...,1,politics
4,CANTDOANYTHINGMONKEYSAREHANUMANINCARNATIONAND...,1,politics
